In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='ru')

In [93]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [3]:
ozon_headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36 OPR/88.0.4412.40',
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'referer': 'http://job.ozon.ru/vacancy/?query='
}

In [4]:
MAIN_URL = "https://job.ozon.ru/vacancy/?query="
API_URL = "https://job.ozon.ru/api/vacancy?query=&limit=10&page=1"

In [5]:
session = requests.Session()
session.headers = ozon_headers

In [6]:
result = requests.get(MAIN_URL)
result

<Response [403]>

In [8]:
print(result.text)

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>

<title>Please Wait... | Cloudflare</title>
  
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" />
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>


<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', fu

### Ошибка Cloudflare, т.е. защита от парсинга запросами
Можно попробовать Selenium или другие варианты вроде undetected_chromedriver

## Использование HeadHunter API

In [46]:
HH_API_URL = 'http://api.hh.ru/'
HH_TOKEN_URL = "https://hh.ru/oauth/token"

In [8]:
CLIENT_ID = "N9N75GOTCRKJAIKK8TVM5UNPI38GC0B39BT0K2Q1AUM3P35LMCPJHQLDC952PH2A"
CLIENT_SECRET = "O2VFGTA91FPTUHDLSUVBOVU3JETH3RE2VBUERKRGP8SK0GGCQ5LCJSIKA5FGAIRB"

In [41]:
token_result = requests.post(HH_TOKEN_URL, data={
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
})

In [44]:
token = token_result.json()['access_token']
token

'IQEK1G2E07TRN3QIMNGM5IB7US6I6NQ4OBJBUML2UI4EIU7VDR1NL32OM2O020LI'

In [45]:
hh_headers = {
    'Authorization': f'Bearer {token}'
}

In [54]:
hh_dictionaries = requests.get(HH_API_URL + 'dictionaries').json()
hh_dictionaries['vacancy_search_order'], hh_dictionaries['vacancy_search_order']

([{'id': 'publication_time', 'name': 'по дате'},
  {'id': 'salary_desc', 'name': 'по убыванию дохода'},
  {'id': 'salary_asc', 'name': 'по возрастанию дохода'},
  {'id': 'relevance', 'name': 'по соответствию'},
  {'id': 'distance', 'name': 'по удалённости'}],
 [{'id': 'publication_time', 'name': 'по дате'},
  {'id': 'salary_desc', 'name': 'по убыванию дохода'},
  {'id': 'salary_asc', 'name': 'по возрастанию дохода'},
  {'id': 'relevance', 'name': 'по соответствию'},
  {'id': 'distance', 'name': 'по удалённости'}])

In [71]:
query_params = {
    'text': 'Ozon',
    'per_page': 50,
    'page': 0,
    'specialization': 1, # IT
    'order_by': 'publication_time',
    'search_field': 'company_name'
}

In [86]:
test_request = requests.get(HH_API_URL + 'vacancies', headers=hh_headers, params=query_params).json()
test_request['pages'], test_request['found']

(9, 414)

In [79]:
test_vacancy = requests.get(HH_API_URL + 'vacancies', headers=hh_headers, params=query_params).json()['items'][0]
test_vacancy.keys()

dict_keys(['id', 'premium', 'name', 'department', 'has_test', 'response_letter_required', 'area', 'salary', 'type', 'address', 'response_url', 'sort_point_distance', 'published_at', 'created_at', 'archived', 'apply_alternate_url', 'insider_interview', 'url', 'adv_response_url', 'alternate_url', 'relations', 'employer', 'snippet', 'contacts', 'schedule', 'working_days', 'working_time_intervals', 'working_time_modes', 'accept_temporary'])

In [92]:
test_result = requests.get(HH_API_URL + f'vacancies/{test_vacancy["id"]}', headers=hh_headers)
test_result.json()['address']

{'city': 'Москва',
 'street': 'Центральный административный округ, Пресненский район, Московский международный деловой центр Москва-Сити',
 'building': None,
 'description': None,
 'lat': 55.749451,
 'lng': 37.542824,
 'raw': 'Москва, Центральный административный округ, Пресненский район, Московский международный деловой центр Москва-Сити',
 'metro': None,
 'metro_stations': []}

In [97]:
job_dicts = []
for page in tqdm(range(test_request['pages'])):
    query_params.update({'page': page})
    items = requests.get(HH_API_URL + 'vacancies', headers=hh_headers, params=query_params).json()['items']

    for item in items:
        job_info = requests.get(HH_API_URL + f'vacancies/{item["id"]}', headers=hh_headers).json()
        job_dict = {
            'title': clear_string(job_info['name']),
            'internal_id' : job_info['id'],
            'url': job_info['alternate_url'],
            'description': clear_string(job_info['description']),
            'responsibilities': None,
            'qualifications': None,
            'company': 'Ozon',
            'location': job_info['address']['raw'] if job_info['address'] else None,
            'department': job_info['department']['name'] if job_info['department'] else None,
            'publish_date': parser.parse(job_info['created_at'])
        }
        job_dicts.append(job_dict)

    time.sleep(0.2) # 200 ms sleep
len(job_dicts)

100%|██████████| 9/9 [01:52<00:00, 12.45s/it]


411

In [98]:
snapshot = pd.DataFrame(filter(lambda x: x is not None, job_dicts))
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,location,department,publish_date
213,Электромеханик,68210224,https://hh.ru/vacancy/68210224,Вам предстоит: · Проведение мероприятий по вып...,None,None,Ozon,None,Ozon Производство,2022-07-27 15:31:47+03:00
391,"Руководитель группы разработки C# / Логистика,...",67716237,https://hh.ru/vacancy/67716237,Ozon имеет разветвленную систему собственных с...,None,None,Ozon,"Москва, Пресненская набережная, 10",Ozon Информационные технологии,2022-07-12 18:53:46+03:00
81,Инженер по автоматизации тестирования Go/Types...,67544619,https://hh.ru/vacancy/67544619,Команда личного кабинета покупателя и избранно...,None,None,Ozon,None,Ozon Информационные технологии,2022-08-05 15:34:51+03:00
13,"Руководитель группы Data Engineer, Ozon Fresh",68596702,https://hh.ru/vacancy/68596702,ЧТО ВАМ ПРЕДСТОИТ: Проектировать и разрабатыва...,None,None,Ozon,None,Ozon fresh,2022-08-10 10:09:10+03:00
116,"Разработчик JavaScript, Поиск и Каталог",68441360,https://hh.ru/vacancy/68441360,Ozon Performance – &quot;martech-стартап&quot;...,None,None,Ozon,"Москва, Пресненская набережная, 10",Ozon Информационные технологии,2022-08-03 14:33:47+03:00


In [99]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype                                
---  ------            --------------  -----                                
 0   title             411 non-null    object                               
 1   internal_id       411 non-null    object                               
 2   url               411 non-null    object                               
 3   description       411 non-null    object                               
 4   responsibilities  0 non-null      object                               
 5   qualifications    0 non-null      object                               
 6   company           411 non-null    object                               
 7   location          214 non-null    object                               
 8   department        411 non-null    object                               
 9   publish_date      411 non-null    datetime6

In [100]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 411 entries, 0 to 410
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
411 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.3 KB


In [101]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [102]:
snapshot.to_csv(f'../data/ozon/{current_date}.csv')
snapshot.to_excel(f'../data/ozon/{current_date}.xlsx', engine='xlsxwriter')